# Model comparison for the classification of moles on the HAM10000 dataset

Disclaimer: you need A LOT of RAM to run this script (>60Gbs)

import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize
from scipy import stats

import itertools

import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## To facilitate variable naming, we will name each model:
### NO segmentation NO hair removal == Model 1
### NO segmentation YES hair removal == Model 2
### YES segmentation YES hair removal == Model 3

## Import datasets

The datasplits are identical, what changes is the preprocessing of the images.

In [ ]:
X_1 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/X_NO_dullrazor_NO_segmentation_128.npy')
y_1 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/y_NO_dullrazor_NO_segmentation_128.npy')

X_2 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/X_hair_removal_NO_segmentation_128.npy')
y_2 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/y_hair_removal_NO_segmentation_128.npy')

X_3 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/X_dullrazor_128_otsu.npy')
y_3 = np.load('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/y_dullrazor_128_otsu.npy')

One-hot encoding labels

In [ ]:
def prepare_labels(labels):
    """Convert string labels to one-hot encoding"""
    #create and fit label encoder
    label_encoder = LabelEncoder()
    numeric_labels = label_encoder.fit_transform(labels)

    #save label encoder classes so we can use them later for interpretation
    label_mapping = dict(zip(label_encoder.classes_,
                            range(len(label_encoder.classes_))))

    #one-hot encoding the numeric-encoded classes
    one_hot_labels = tf.keras.utils.to_categorical(numeric_labels)

    #Print mapping for verification
    print("Label mapping:")
    for label, idx in label_mapping.items():
        print(f"{label}: {idx}")

    return one_hot_labels, label_encoder

In [ ]:
y_1_encoded, label_encoder = prepare_labels(y_1)
y_2_encoded, _ = prepare_labels(y_2)
y_3_encoded, _ = prepare_labels(y_3)

Label mapping:
akiec: 0
bcc: 1
bkl: 2
df: 3
mel: 4
nv: 5
vasc: 6
Label mapping:
akiec: 0
bcc: 1
bkl: 2
df: 3
mel: 4
nv: 5
vasc: 6
Label mapping:
akiec: 0
bcc: 1
bkl: 2
df: 3
mel: 4
nv: 5
vasc: 6


In [ ]:
#define splits
##---Data splitting: we want a 75, 20, 5 train/test/validation split
def datasplits(X, y_encoded):

    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y_encoded,
        test_size=0.20,
        random_state=42, #for reproductibility
        stratify=y_encoded
    )

      #Then split remaining data into train and validation (val is 5% of total)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val,
        test_size=0.0625,  #0.05/0.80 to get 5% of total data
        random_state=42,
        stratify=y_train_val
    )
    return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
#all of the y_train, test and val are the same
X_1_train, X_1_test, X_1_val, y_train, y_test, y_val = datasplits(X_1, y_1_encoded)
X_2_train, X_2_test, X_2_val, _, _, _ = datasplits(X_2, y_2_encoded)
X_3_train, X_3_test, X_3_val, _, _, _ = datasplits(X_3, y_3_encoded)

## Model evaluation function - We use McNemar's test

In [ ]:
def mcnemar_test(y_true, pred1, pred2):
    """
    Perform McNemar's test to compare two models.
    Returns test statistic, p-value, and contingency table.
    """
    #Creation of the contingency table
    correct1 = pred1 == y_true
    correct2 = pred2 == y_true

    #Contingency table values
    b = np.sum(~correct1 & correct2)  #model1 wrong, model2 right
    c = np.sum(correct1 & ~correct2)  #model1 right, model2 wrong

    #Calculate test statistic and p-value
    statistic = (abs(b - c) - 1)**2 / (b + c)
    p_value = stats.chi2.sf(statistic, df=1)

    contingency_table = {
        'model1_right_model2_right': np.sum(correct1 & correct2),
        'model1_right_model2_wrong': c,
        'model1_wrong_model2_right': b,
        'model1_wrong_model2_wrong': np.sum(~correct1 & ~correct2)
    }

    return statistic, p_value, contingency_table

In [ ]:
def compare_models(models, X_test_dict, y_test, label_encoder, model_names=None):
    """
    Compare multiple models with statistical testing.
    """
    if model_names is None:
        model_names = [f'Model_{i}' for i in range(len(models))]

    # Perform pairwise statistical tests
    pairwise_results = []
    for i, model1 in enumerate(models):
        for j in range(i + 1, len(models)):  #Start j from i+1 to avoid redundant comparisons
            model2 = models[j]
            #get correct datasplit
            X_test_1 = X_test_dict[model_names[i]]
            X_test_2 = X_test_dict[model_names[j]]

            #Model predictions
            model_pred_1 = np.argmax(model1.predict(X_test_1), axis=1)
            model_pred_2 = np.argmax(model2.predict(X_test_2), axis=1)
            return model_pred_1, model_pred_2
            statistic, p_value, contingency = mcnemar_test(
                y_test,
                model_pred_1,
                model_pred_2
            )

            result = {
                'Model 1': model_names[i],
                'Model 2': model_names[j],
                'Test Statistic': statistic,
                'P-value': p_value,
                'Significant': p_value < 0.05,
                **contingency
            }
            pairwise_results.append(result)

    statistical_tests = pd.DataFrame(pairwise_results)
    return statistical_tests


## Load models

In [ ]:
#model 1
frozen_1 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_no_dullrazor_no_segmentation.keras')
fine_tuned_1 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Fine_tuned_model/densenet201_ph2_128_no_dullrazor_no_segmentation.keras')

#model 2
frozen_2 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_no_segmentation.keras')
fine_tuned_2 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_no_segmentation.keras')

#model 3
frozen_3 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Fine_tuned_model/densenet201_ph2_128_dullrazor_segmented.keras')
fine_tuned_3 = load_model('/content/drive/MyDrive/Project 36100 - Andrea, Monika, Yamuna/Assignment Stage 2/Frozen_model/densenet201_ph1_128_dullrazor_segmented.keras')

In [ ]:
#compare models
models = [frozen_1, fine_tuned_1]
model_names = ["F_1", "FT_1"]
X_test_dict = {"F_1":X_1_test, "FT_1":X_1_test}

model_pred_1, model_pred_2 = compare_models(models, X_test_dict, y_test, label_encoder, model_names)

In [ ]:
y_true = np.argmax(y_test, axis=1)
mcnemar_test(y_true, model_pred_1, model_pred_2)

In [ ]:
#compare models
models = [frozen_1, fine_tuned_1]
model_names = ["F_1", "FT_1"]
X_test_dict = {"F_1":X_1_test, "FT_1":X_1_test}

comparison_df, detailed_metrics, statistical_tests = compare_models(models, X_test_dict, y_test, label_encoder, model_names)
# Print the basic comparison
print("\nModel Comparison Summary:")
print(comparison_df.to_string(index=False))

# Print statistical test results
print("\nPairwise Statistical Tests:")
print(statistical_tests.to_string(index=False))